In [1]:
%matplotlib inline
import pandas as pd
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARIMA
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import numpy as np

def root_mean_squared_error(t,p):
    return np.sqrt(mean_squared_error(t,p))

In [7]:
train_raw = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
sample_submission = pd.read_csv("../input/sampleSubmission.csv")
import datetime

In [58]:
sample_submission.head()

,Id,Incidents
0,2018-12-01 06:00:00_hex_000,0
1,2018-12-01 14:00:00_hex_000,0
2,2018-12-01 22:00:00_hex_000,0
3,2018-12-02 06:00:00_hex_000,0
4,2018-12-02 14:00:00_hex_000,0


In [8]:
train_raw

,Date,hex_000,hex_001,hex_002,hex_003,hex_004,hex_005,hex_006,hex_007,hex_008,hex_009,hex_010,hex_011,hex_012,hex_013,hex_014,hex_015,hex_016,hex_017,hex_018,hex_019,hex_020,hex_021,hex_022,hex_023,hex_024,hex_025,hex_026,hex_027,hex_028,hex_029,hex_030,hex_031,hex_032,hex_033,hex_034,hex_035,hex_036,hex_037,hex_038,...,hex_279,hex_280,hex_281,hex_282,hex_283,hex_284,hex_285,hex_286,hex_287,hex_288,hex_289,hex_290,hex_291,hex_292,hex_293,hex_294,hex_295,hex_296,hex_297,hex_298,hex_299,hex_300,hex_301,hex_302,hex_303,hex_304,hex_305,hex_306,hex_307,hex_308,hex_309,hex_310,hex_311,hex_312,hex_313,hex_314,hex_315,hex_316,hex_317,hex_318
0,2015-01-01 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,11.0,2.0,1.0,1.0,0.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015-01-01 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-01-01 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,0.0,0.0,1.0,4.0,4.0,0.0,4.0,0.0,7.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015-01-02 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,8.0,0.0,4.0,0.0,1.0,0.0,0.0,2.0,0.0,5.0,2.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015-01-02 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2015-01-02 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
6,2015-01-03 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2015-01-03 14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
8,2015-01-03 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,5.0,4.0,1.0,1.0,0.0,2.0,0.0,3.0,3.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2015-01-04 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,4.0,1.0,0.0,2.0,0.0,0.0,0.0,2

In [10]:
train_raw['Date'] = pd.to_datetime(pd.Series(train_raw['Date']))


In [11]:
train_raw['hour'] = train_raw['Date'].dt.hour
train_raw['dayofweek'] = train_raw['Date'].dt.dayofweek


In [12]:
train_raw['quarter'] = train_raw['Date'].dt.quarter
train_raw['month'] = train_raw['Date'].dt.month
train_raw['year'] = train_raw['Date'].dt.year
train_raw['dayofyear'] = train_raw['Date'].dt.dayofyear
train_raw['dayofmonth'] = train_raw['Date'].dt.day
train_raw['weekofyear'] = train_raw['Date'].dt.weekofyear

In [25]:
train_raw['year'].value_counts()

2016    1098
2015    1095
2017    1095
2018    1002
Name: year, dtype: int64

In [2]:
valid = np.zeros((1002,328))
for i in range(len(train_raw)):
    c = 0
    if train_raw.iloc[i,324] == 2018:
        valid[c,:] = train_raw.iloc[i,:]
        c += 1

In [41]:
train_raw = train_raw.reset_index('Date')

In [38]:
validx = train_raw.index.year==2018

In [40]:
train_raw[~validx].shape

(3288, 327)

In [46]:
train = pd.melt(train_raw, id_vars=['Date'], value_vars=[i for i in train_raw.columns if 'hex_' in i])

In [56]:
validx =  train['Date'].dt.year==2018
#validx
train[~validx].head()

,Date,variable,value,hour,day
0,2015-01-01 06:00:00,hex_000,0.0,6,1
1,2015-01-01 14:00:00,hex_000,0.0,14,1
2,2015-01-01 22:00:00,hex_000,0.0,22,1
3,2015-01-02 06:00:00,hex_000,0.0,6,2
4,2015-01-02 14:00:00,hex_000,0.0,14,2


In [55]:
# create features
train['hour'] = train['Date'].dt.hour
train['day'] = train['Date'].dt.day
train.iloc[:,2:].head()

,value,hour,day
0,0.0,6,1
1,0.0,14,1
2,0.0,22,1
3,0.0,6,2
4,0.0,14,2


In [59]:
#train.pivot(index='Date', columns='variable', values='value')

In [61]:
train.index = train['Date'].astype(str)+'_'+train['variable']

In [62]:
train.head()

,Date,variable,value,hour,day
2015-01-01 06:00:00_hex_000,2015-01-01 06:00:00,hex_000,0.0,6,1
2015-01-01 14:00:00_hex_000,2015-01-01 14:00:00,hex_000,0.0,14,1
2015-01-01 22:00:00_hex_000,2015-01-01 22:00:00,hex_000,0.0,22,1
2015-01-02 06:00:00_hex_000,2015-01-02 06:00:00,hex_000,0.0,6,2
2015-01-02 14:00:00_hex_000,2015-01-02 14:00:00,hex_000,0.0,14,2
